In [1]:
import pandas as pd
import os

# Caminhos dos arquivos enviados
file_paths = [
    "/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/data/raw/indicadores.csv",
    "/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/data/raw/indicadores.csv",
    "/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/data/raw/indicadores.csv",
    "/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/data/raw/indicadores.csv"
]

# Carregar os arquivos CSV
dataframes = []
for file in file_paths:
    try:
        df = pd.read_csv(file, sep=";", encoding="utf-8")
        dataframes.append(df)
    except Exception as e:
        print(f"Erro ao carregar {file}: {e}")

# Unir os dados em um único DataFrame
df_pcm = pd.concat(dataframes, ignore_index=True)

# Exibir as primeiras linhas para análise
df_pcm.head()


,OS,SETOR,COD_MAQUINA,MAQUINA,DIVISAO,EQUIPAMENTO,COD_EQUIPAMENTO,NOME,DATA_ABERTURA,DATA_PARADA,...,MOTIVO,OFICINA,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,1,IMPRESSAO,203.0,IMPRESSORA 203 FLEXO TECH 8 CORES,GRUPO IMPRESSOR,NaN,NaN,SERGIO BATISTA CARDOSO,15/02/2018 17:33,NaN,...,CORRETIVA DA PREVENTIVA,Mecanico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,EXTRUSAO,109.0,109 MegaSteel monocamada,EXTRUSORA,NaN,NaN,PROJEDATA - JEFERSON APARECIDO DOS REIS,28/05/2018 19:14,NaN,...,CORRETIVA DA PREVENTIVA,Mecanico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,EXTRUSAO,109.0,109 MegaSteel monocamada,EXTRUSORA,NaN,NaN,MURILO FLORENCIO DE SANTANA,29/05/2018 17:14,NaN,...,CORRETIVA DA PREVENTIVA,Mecanico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,DIRETORIA,NaN,NaN,NaN,NaN,NaN,ALAN MANI,31/05/2018 18:13,31/05/2018 18:11,...,CORRETIVA,Eletrico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,EXTRUSAO,115.0,115 - Carnevalli Coex 3 camadas,CONJUNTO EXTRUSOR,NaN,NaN,MANUTENCAO,04/09/2018 14:19,NaN,...,CORRETIVA DA PREVENTIVA,Mecanico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Converter colunas de data para formato datetime
df_pcm["DATA_ABERTURA"] = pd.to_datetime(df_pcm["DATA_ABERTURA"], errors="coerce", format="%d/%m/%Y %H:%M")
df_pcm["DATA_PARADA"] = pd.to_datetime(df_pcm["DATA_PARADA"], errors="coerce", format="%d/%m/%Y %H:%M")
df_pcm["DATA_FECHAMENTO"] = pd.to_datetime(df_pcm["DATA_FECHAMENTO"], errors="coerce", format="%d/%m/%Y %H:%M")

# Criar coluna de tempo de atendimento (diferença entre abertura e fechamento)
df_pcm["TEMPO_ATENDIMENTO"] = (df_pcm["DATA_FECHAMENTO"] - df_pcm["DATA_ABERTURA"]).dt.total_seconds() / 3600

# Remover colunas desnecessárias (Unnamed e outras irrelevantes)
df_pcm = df_pcm.loc[:, ~df_pcm.columns.str.contains("Unnamed")]

# Exibir estatísticas do tempo de atendimento
df_pcm["TEMPO_ATENDIMENTO"].describe()



count    17476.000000
mean      1312.025837
std       2201.726280
min      -1102.100000
25%        121.466667
50%        571.716667
75%       1700.716667
max      37873.366667
Name: TEMPO_ATENDIMENTO, dtype: float64

In [8]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Definir caminho da pasta dos arquivos
data_path = "/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/data/raw/"

# Listar arquivos CSV na pasta
arquivos = [f for f in os.listdir(data_path) if f.endswith('.csv')]

# Carregar e unir os arquivos CSV
df_list = [pd.read_csv(os.path.join(data_path, f), encoding='utf-8') for f in arquivos]
df = pd.concat(df_list, ignore_index=True)

# Converter colunas de data para datetime
df['DATA_ABERTURA'] = pd.to_datetime(df['DATA_ABERTURA'], errors='coerce')
df['DATA_FECHAMENTO'] = pd.to_datetime(df['DATA_FECHAMENTO'], errors='coerce')

# Remover valores negativos e inconsistentes
df = df[(df['DATA_FECHAMENTO'] >= df['DATA_ABERTURA'])]

# Criar coluna de tempo de atendimento
df['TEMPO_ATENDIMENTO'] = (df['DATA_FECHAMENTO'] - df['DATA_ABERTURA']).dt.total_seconds() / 3600  # Em horas

# Estatísticas gerais
tempo_atendimento_stats = df['TEMPO_ATENDIMENTO'].describe()

# Análise por período
df['MES'] = df['DATA_ABERTURA'].dt.to_period('M')
df['TRIMESTRE'] = df['DATA_ABERTURA'].dt.to_period('Q')
df['SEMESTRE'] = df['DATA_ABERTURA'].dt.to_period('2Q')
df['ANO'] = df['DATA_ABERTURA'].dt.to_period('Y')

# Agrupar por setor e calcular estatísticas
df_setor = df.groupby('SETOR')['TEMPO_ATENDIMENTO'].describe()

# Identificar equipamentos com mais falhas
df_equipamentos = df['COD_EQUIPAMENTO'].value_counts()

# Criar gráficos
plt.figure(figsize=(12, 6))
sns.boxplot(x='SETOR', y='TEMPO_ATENDIMENTO', data=df)
plt.xticks(rotation=45)
plt.title("Tempo de Atendimento por Setor")
plt.show()

plt.figure(figsize=(12, 6))
df['MES'].value_counts().sort_index().plot(kind='bar', color='blue')
plt.title("Número de OS por Mês")
plt.show()

# Salvar relatório em CSV
df_setor.to_csv("/workspaces/ANALISES-DE-DADOS/Analise de PCM/20250320/projeto_pcm/reports/relatorio.csv")

df.head()



ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2
